# Detecção de Objetos e Segmentação de Instâncias utilizando o TensorFlow

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/beonclaro/Campus-Mobile/blob/master/Reconhecimento%20de%20imagens/%5BCampus_Mobile%5D_Object_Detection_Tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Acessar no Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/beonclaro/Campus-Mobile/blob/master/Reconhecimento%20de%20imagens/%5BCampus_Mobile%5D_Object_Detection_Tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver código fonte no GitHub</a>
</td></table>

Esse notebook foi construído como material de suporte para o [webinar sobre práticas de Inteligência Artificial no reconhecimento de imagens](https://www.youtube.com/watch?v=NUfQxBqnnp4), transmitido através do canal do [Programa Campus Mobile no Youtube](https://www.youtube.com/channel/UCnyoGz2f3gcta37wOKkiEKw).

> **Referências**:

[Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)

[Object Detection API Demo](https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb)

[Detecção de objetos utilizando o TFHub](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb)




# Setup

**Importante**: Se você esta rodando este notebook na sua máquina local, por favor siga as [instruções de instalação](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). Esse notebook inclui apenas o necessário para rodar a demo no Google Colab.


### Instalação

In [0]:
!pip install -U --pre tensorflow=="2.*"

Certifique-se de que a biblioteca `pycocotools` esta instalada.

In [0]:
!pip install pycocotools

Clonando o diretório `https://github.com/tensorflow/models`

In [0]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compilando dependências como o protobufs e instalando o pacote object_detection

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [0]:
%%bash 
cd models/research
pip install .

### Importando bibliotecas

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

from PIL import Image
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import time

from IPython.display import display

Importando o módulo object detection

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Preparação do modelo 

Neste tutorial abordaremos o processo de inferência utilizando um dos modelos disponíveis no [detection model zoo do Tensorflow](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).

O processo de treinamento não será abordado em detalhe neste tutorial. Caso tenha interesse em entender mais sobre como treinar um modelo de detecção de objetos, verifique as referências abaixo:

[Transferência de Aprendizado utilizando o Tensorflow](https://www.tensorflow.org/tutorials/images/transfer_learning)

[Detecção de Objetos utilizando o seu próprio dataset](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md)

## Carregando um modelo treinado previamente

In [0]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

## Carregando o mapeamento para o nome das classes
A saída da rede neural convolucional é um número, sendo assim precisamos saber o mapeamento do número para a classe. Por exemplo, se a saída da rede for `5`, precisamos saber que isso corresponde à classe `avião`.

In [0]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Imagens de teste

In [0]:
def download_and_resize_image(url, new_width=256, new_height=256):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
 
  return filename

In [0]:
image_urls = ["https://www.institutonetclaroembratel.org.br/wp-content/uploads/2019/09/9_1.jpg",
              "https://www.institutonetclaroembratel.org.br/wp-content/uploads/2019/09/5.jpg",
             "https://www.institutonetclaroembratel.org.br/wp-content/uploads/2019/09/7.jpg",
             "https://www.institutonetclaroembratel.org.br/wp-content/uploads/2019/09/6.jpg",
              "https://www.institutonetclaroembratel.org.br/wp-content/uploads/2019/09/10.jpg"]

for image_url in image_urls:
  image_path = download_and_resize_image(image_url, 800, 800)
  image_np = np.array(Image.open(image_path))
  display(Image.fromarray(image_np))

# Detecção de objetos

Carregando um modelo de detecção de objetos:

In [0]:
#model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
model_name = 'ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
detection_model = load_model(model_name)

Verifique as dimensões de entrada do modelo, ele espera um conjunto de imagens com três canais de cores do tipo uint8:

In [0]:
print(detection_model.inputs)

O modelo retorna muitas informações na saída:

In [0]:
detection_model.output_dtypes

In [0]:
detection_model.output_shapes

Abaixo temos uma função que realiza a inferência em uma imagem utilizando o modelo que foi previamente carregado.

In [0]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Abaixo temos uma função que chama a função que realiza a inferência e desenha os "boxes" na localização e com as classes que foram inferidas na imagem de entrada.

In [0]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

Rodando a inferência para um conjunto de imagens

In [0]:
#for image_path in TEST_IMAGE_PATHS:
#  show_inference(detection_model, image_path)
for image_url in image_urls:
  start_time = time.time()
  image_path = download_and_resize_image(image_url, 640, 640)
  show_inference(detection_model, image_path)
  end_time = time.time()
  print("Inference time: {} sec".format(end_time-start_time))

## Segmentação de instâncias

Nesta etapa vamos substituir o modelo de detecção de objetos por um modelo que realiza a segmentação de instâncias.

In [0]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model("mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28")

O modelo de segmentação de instâncias inclui uma saída com as as máscaras detectadas, já que a classificação neste caso é pixel à pixel.

In [0]:
masking_model.output_shapes

In [0]:
for image_url in image_urls:
  start_time = time.time()
  image_path = download_and_resize_image(image_url, 640, 640)
  show_inference(masking_model, image_path)
  end_time = time.time()
  print("Inference time: {} sec".format(end_time-start_time))